# Gets the names and images by IDs from a champion page

In [56]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import json


In [57]:
def extract_image_tags(html_content):
    try:
        # Parse the HTML content
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all image tags
        body = soup.find('body')
        img_tags = body.find_all('img')

        # Return the image tags
        return img_tags
    except Exception as e:
        return f"An error occurred: {e}"
   
steps = 10
sleep_time = 0.5

def get_data(url):
    try:
        # Create a driver to browse URL
        driver = webdriver.Firefox()
        driver.get(url)

        height = driver.execute_script("return document.body.scrollHeight")
        #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        for i in range(steps):
            to_scroll = height * i / steps
            driver.execute_script(f"window.scrollTo(0, {to_scroll});")
            time.sleep(sleep_time)
            height = driver.execute_script("return document.body.scrollHeight")
        
        
        html = driver.page_source
        return html
    except Exception as e:
        print(e)
        return f"get_data(): An error occurred: {e}"
    finally:
        driver.quit()

Scroll through page and load all images from lolalytics site. 

In [58]:
id_filename = 'ids.json'

def dict_to_json_file(id_data, file_name):
    try:
        if id_data: # Writing to an existing file
            # Open the file in write mode
            with open(file_name, 'w') as file:
                # Write the dictionary to the file in JSON format
                json.dump(id_data, file, indent=4)
            print(f"Dictionary successfully written to {file_name}")
        else: # Creating a new file
            with open(file_name, 'w') as file:
                # Write the dictionary to the file in JSON format
                json.dump(id_data, file, indent=4)
            print(f"Created {file_name}")
    except Exception as e:
        print(f"dict_to_json_file(): An error occurred: {e}")

        
def json_file_to_dict(file_name):
    try:
        # Open the file for reading
        with open(file_name, 'r') as file:
            # Read and convert the JSON data into a Python dictionary
            data = json.load(file)
            return data
    except Exception as e:
        print(f"json_file_to_dict(): An error occurred: {e}")
        dict_to_json_file({}, id_filename)
        return {}

ids_json = json_file_to_dict(id_filename)


In [59]:
def switch_key_and_val(data):
    
    # Switch the keys and 'name' values
    new_data = {}
    for category_key, category_val in data.items():
        new_data[category_key] = {value['name']: key for key, value in category_val.items()}
        # Write the modified data back to a JSON file
    print("The keys and name values have been switched.")
    return new_data

def category(url):
    # Split the URL by '/' and get the second to last
    try:
        cat = url.split('/')[-2]
        # Remove all the numbers
        cat = ''.join([char for char in cat if not char.isdigit()])
        return cat
    except Exception as e:
        print(f"category(): An error occurred: {e}")
        return None
    
def extract_img_attributes(img_tags):
    # Dict to store image data
    id_data = dict()
    # Iterate over each img tag
    for img in img_tags:
        # Extract attributes
        alt = img.get('alt', '')  # Default to empty string if alt is not present
        src = img.get('src', '')  # Default to empty string if src is not present
        data_id = img.get('data-id', '')  # Default to empty string if data-id is not present
        if not data_id:
            data_id = src.split('/')[-1].split('.')[0] # Returns the string after the last '/' and first '.'
        if not (alt and data_id and src):
            continue
        # Find the category of the img
        img_category = category(src)
        if not img_category in id_data:
            id_data[img_category] = {}
        # Append the extracted data to the dict
        id_data[img_category][data_id] = {'name': alt, 'img': src}

    return id_data

def merge_dict(dict1, dict2):
    for key, values in dict2.items():
        if key not in dict1:
            dict1[key] = values
        elif type(dict1[key]) is dict: # Shallow copy
            dict1[key] = {**dict2[key], **dict1[key]}
    return dict1

# Specific dict clean up from beta site 
def clean_up():
    clean = []
    for key in ids_json['item'].keys():
        if '_' in key:
            clean.append(key)
    for key in clean:
        #ids_json['item'].pop(key)
        del ids_json['item'][key]
    
    clean = []
    for key in ids_json['champx'].keys():
        if any(char.isalpha() for char in key):
            clean.append(key)
    for key in clean:
        #ids_json['champx'].pop(key)
        del ids_json['champx'][key]

In [60]:
champions_all = [
    "aatrox", "ahri", "akali", "alistar", "amumu", "anivia", "annie", "aphelios",
    "ashe", "aurelionsol", "azir", "bard", "blitzcrank", "brand", "braum",
    "caitlyn", "camille", "cassiopeia", "chogath", "corki", "darius", "diana",
    "drmundo", "draven", "ekko", "elise", "evelynn", "ezreal", "fiddlesticks",
    "fiora", "fizz", "galio", "gangplank", "garen", "gnar", "gragas", "graves",
    "gwen", "hecarim", "heimerdinger", "illaoi", "irelia", "ivern", "janna",
    "jarvaniv", "jax", "jayce", "jhin", "jinx", "kaisa", "kalista", "karma",
    "karthus", "kassadin", "katarina", "kayle", "kayn", "kennen", "khazix",
    "kindred", "kled", "kogmaw", "leblanc", "leesin", "leona", "lillia",
    "lissandra", "lucian", "lulu", "lux", "malphite", "malzahar", "maokai",
    "masteryi", "missfortune", "mordekaiser", "morgana", "nami", "nasus",
    "nautilus", "neeko", "nidalee", "nocturne", "nunu", "olaf", "orianna",
    "ornn", "pantheon", "poppy", "pyke", "qiyana", "quinn", "rakan", "rammus",
    "reksai", "rell", "renata", "renekton", "rengar", "riven", "rumble", "ryze",
    "samira", "sejuani", "senna", "seraphine", "sett", "shaco", "shen", "shyvana",
    "singed", "sion", "sivir", "skarner", "sona", "soraka", "swain", "sylas",
    "syndra", "tahmkench", "taliyah", "talon", "taric", "teemo", "thresh",
    "tristana", "trundle", "tryndamere", "twistedfate", "twitch", "udyr",
    "urgot", "varus", "vayne", "veigar", "velkoz", "vex", "vi", "viego",
    "viktor", "vladimir", "volibear", "warwick", "wukong", "xayah", "xerath",
    "xinzhao", "yasuo", "yone", "yorick", "yuumi", "zac", "zed", "ziggs",
    "zilean", "zoe", "zyra"
]

champions = ['ahri']
isFirst = True
for champion in [champions[0]] + champions_all:
#for champion in champions:
    if isFirst:
        url = f"https://beta.lolalytics.com/lol/{champion}/build"
        isFirst = False
    else:
        url = f"https://lolalytics.com/lol/{champion}/build"
    html = get_data(url)
    images = extract_image_tags(html)
    if not images:
        print(f'Could not find data for {champion}')
        break
    # Make a json file if it doesn't exist
    merge_dict(ids_json, extract_img_attributes(images))
    # Specific cleanup from beta site data
    clean_up() 
    dict_to_json_file(ids_json, id_filename)



Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written to ids.json
Dictionary successfully written 

In [61]:
def reverse_and_add(json):
    # Reverse key-value pairs. Make sure values are unique and hashable.
    reversed_pairs = {value: key for key, value in json.items()}
    
    # Check for potential conflicts (same key in original and reversed)
    for key in reversed_pairs:
        if key in json and reversed_pairs[key] != json[key]:
            raise ValueError("Conflict detected: Cannot safely add reversed pairs to the original dictionary.")
    
    # Merge the original dictionary with the reversed pairs
    updated_dict = {**json, **reversed_pairs}
    return updated_dict

names_json = switch_key_and_val(ids_json)
for key, val in names_json.items():
    names_json[key] = reverse_and_add(val)

dict_to_json_file(names_json, 'names.json')



The keys and name values have been switched.
Dictionary successfully written to names.json
